# Be able to investigate client requirements for data analysis
# 2.4 Quantitative data analysis

* mean
* median
* standard deviation
* range

## Set up

In [1]:
try:
    import micropip
    await micropip.install(["pyoliteutils", "textblob"])
except:
    pass

In [2]:
from pyoliteutils import *
import pandas as pd

## Crowd / Attendance Data

In [4]:
crowds = pd.read_csv("../data/eagles/crowds.csv")
## https://datascienceparichay.com/article/pandas-extract-year-from-datetime-column/
crowds

Date     Opponents  Attendance
0    02/09/1984      Rochdale        1425
1    09/09/1984        Fulham        1145
2    23/09/1984       Salford        1159
3    14/10/1984       Runcorn        1076
4    28/10/1984      Bridgend         826
..          ...           ...         ...
583  24/07/2022    Workington         644
584  07/08/2022    Whitehaven         727
585  12/08/2022       Halifax         569
586  02/09/2022  Featherstone         912
587  14/09/2022      Dewsbury         604

[588 rows x 3 columns]

## Get Usable Date Information

In [13]:
crowds['Date'] = pd.to_datetime(crowds['Date'], format="%d/%m/%Y")
crowds['Year'] = crowds['Date'].dt.year
crowds['Day'] = crowds['Date'].dt.day_name()
crowds['month'] = crowds['Date'].dt.month
crowds['month_name'] = crowds['Date'].dt.month_name()
crowds

Date     Opponents  Attendance  Year  month month_name        Day
0   1984-09-02      Rochdale        1425  1984      9  September     Sunday
1   1984-09-09        Fulham        1145  1984      9  September     Sunday
2   1984-09-23       Salford        1159  1984      9  September     Sunday
3   1984-10-14       Runcorn        1076  1984     10    October     Sunday
4   1984-10-28      Bridgend         826  1984     10    October     Sunday
..         ...           ...         ...   ...    ...        ...        ...
583 2022-07-24    Workington         644  2022      7       July     Sunday
584 2022-08-07    Whitehaven         727  2022      8     August     Sunday
585 2022-08-12       Halifax         569  2022      8     August     Friday
586 2022-09-02  Featherstone         912  2022      9  September     Friday
587 2022-09-14      Dewsbury         604  2022      9  September  Wednesday

[588 rows x 7 columns]

## Saving for later

In [14]:
crowds.to_csv("../data/eagles/crowds_clean.csv")

## Other useful Opponent / Match Data?

### Location of opponents, and distance to Sheffield

#### Possible Data Sources : 
- https://api.postcodes.io/places?q=[query]
- Nominatum / GeoPy
    - https://geopy.readthedocs.io/en/stable/
    - https://nominatim.org/release-docs/latest/api/Search/ 
    - https://medium.com/@gopesh3652/geocoding-with-python-using-nominatim-a-beginners-guide-220b250ca48d 



### Weather in Sheffield

Could do daily weather at the correct stadium location but monthly for Sheffield's Weather station will be easier at first

#### Possible Data Sources : 

- https://www.metoffice.gov.uk/research/climate/maps-and-data/historic-station-data saved as [text](../data/eagles/sheffield_montly_weather.csv)
- https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=39.099724&lon=-94.578331&dt=1643803200&appid=b112bddb3ca3876644b7c695768ae96d

In [15]:
sheffield_monthly_weather = pd.read_csv("../data/eagles/sheffield_monthly_weather.csv")
sheffield_monthly_weather

yyyy  mm tmax degC tmin degC af days rain mm               sun hours
0     1883   1       6.3       1.7       6   122.1                     ---
1     1883   2         8       2.8       2    69.8                     ---
2     1883   3       4.8      -1.6      23    29.6                     ---
3     1883   4      12.2       3.8       2      74                     ---
4     1883   5      14.7       6.2       0    31.2                     ---
...    ...  ..       ...       ...     ...     ...                     ...
1689  2023  10      14.9       8.9       0   203.2      77.5#  Provisional
1690  2023  11       9.4         5       2   108.2      68.1#  Provisional
1691  2023  12       9.1  *    4.7       5   174.3  *   31.5#  Provisional
1692  2024   1         7       2.6       6    72.4      54.9#  Provisional
1693  2024   2      10.2       4.8       2    92.4      42.0#  Provisional

[1694 rows x 7 columns]

## Stadium Data

In [16]:
stadiums = pd.read_csv("../data/eagles/stadiums.csv")
stadiums

Stadium  Postcode
0             OWLERTON    S6 2DE
1             MILLMOOR   S60 1HB
2           HEADINGLEY   LS6 3BR
3         BRAMALL LANE    S2 4QX
4         HILLSBOROUGH    S6 1SW
5              OAKWELL   S71 1ET
6            BELLE VUE   DN4 5DX
7           SALTERGATE   S40 4SX
8         TATTERSFIELD   DN4 5JW
9           THRUM HALL   HX1 4LG
10          DON VALLEY    S9 3TL
11            KEEPMOAT   DN4 5JW
12          HALLAM S P    S9 3HL
13               O L P    S9 3TL
14   CARDIFF ARMS PARK  CF10 1JA
15        CLIFTON LANE   S65 2AA
16            WOODBURN    S9 3HL
17            MOUNT ST  SO14 5FP
18  MARY CASTLE’S PARK   DN2 5QB
19    POST OFFICE ROAD   WF7 5EP

## Adding the Location

In [17]:
import postcodes_io_api
api  = postcodes_io_api.Api()

def get_latlong(postcode):
    latitude = None
    longitude = None
    
    data = api.get_postcode(postcode)
    #print("data", data)
    if data["status"] != 200 :
        data = api.get_outcode(postcode)
    
    if data["status"] == 200 :        
        if "latitude" in data["result"]:
            latitude = data["result"]["latitude"]
            longitude = data["result"]["longitude"]
    #print("latitude, longitude", latitude, longitude)
    return latitude, longitude
    

def get_latlongs(df):
    if ("Latitude" not in df) and ("Postcode" in df):    
      try:
        df[["Latitude", "Longitude"]] = df.apply(
            lambda row: get_latlong(row["Postcode"]), axis=1, result_type="expand"
        )
      except Exception as e:
          print('Postcode Conversion failed : '+ str(e))
    return df

In [19]:
stadiums = get_latlongs(stadiums)
stadiums


Stadium  Postcode   Latitude  Longitude
0             OWLERTON    S6 2DE  53.406031  -1.493303
1             MILLMOOR   S60 1HB  53.428758  -1.369397
2           HEADINGLEY   LS6 3BR  53.816081  -1.580617
3         BRAMALL LANE    S2 4QX  53.371341  -1.469862
4         HILLSBOROUGH    S6 1SW  53.410844  -1.500859
5              OAKWELL   S71 1ET  53.552266  -1.468631
6            BELLE VUE   DN4 5DX  53.517626  -1.108750
7           SALTERGATE   S40 4SX  53.238963  -1.434745
8         TATTERSFIELD   DN4 5JW  53.509088  -1.113845
9           THRUM HALL   HX1 4LG  53.721864  -1.884001
10          DON VALLEY    S9 3TL  53.395919  -1.425961
11            KEEPMOAT   DN4 5JW  53.509088  -1.113845
12          HALLAM S P    S9 3HL  53.389960  -1.431634
13               O L P    S9 3TL  53.395919  -1.425961
14   CARDIFF ARMS PARK  CF10 1JA  51.479971  -3.183833
15        CLIFTON LANE   S65 2AA  53.429825  -1.348792
16            WOODBURN    S9 3HL  53.389960  -1.431634
17            MOUNT ST  SO14 5FP  50.906530  -1.390823
18  MARY CASTLE’S PARK   DN2 5QB  53.532740  -1.079900
19    POST OFFICE ROAD   WF7 5EP  53.677887  -1.355116

In [20]:
stadiums.to_csv("../data/eagles/stadiums_with_latlong.csv")

## Survey Data

In [21]:
questionnaire = pd.read_csv("../data/eagles/questionnaire.csv")
questionnaire

Timestamp                                           Are you:  \
0     9/1/2021 14:05:26                   A player or member of club staff   
1      9/2/2021 9:23:44   A current Sheffield Eagles fan who attends games   
2      9/2/2021 9:25:42   A current Sheffield Eagles fan who attends games   
3     9/2/2021 10:05:39   A current Sheffield Eagles fan who attends games   
4     9/2/2021 10:53:30   A current Sheffield Eagles fan who attends games   
..                  ...                                                ...   
122   9/27/2021 8:00:25                   A player or member of club staff   
123  9/27/2021 10:13:35  A Sheffield Eagles fan who no longer attends g...   
124  9/27/2021 12:35:21   A current Sheffield Eagles fan who attends games   
125  9/27/2021 22:27:20                                   A former player    
126   11/4/2021 0:19:42                   A player or member of club staff   

    When did you first attend a Sheffield Eagles game?  \
0                                           2011- 2021   
1                                          1984 - 1990   
2                                          1991 - 2000   
3                                          1984 - 1990   
4                                          1991 - 2000   
..                                                 ...   
122                                                NaN   
123                                        1991 - 2000   
124                                        1991 - 2000   
125                                        2001 - 2010   
126                                        2001 - 2010   

                          Would you class yourself as:  \
0    Someone who attends games home or away on a re...   
1    Someone who has a season ticket and attends al...   
2    Someone who has a season ticket and attends al...   
3    Someone who attends games home or away on a re...   
4    Someone who has a season ticket and attends al...   
..                                                 ...   
122                         A current or former player   
123                         A current or former player   
124  Someone who has a season ticket and attends al...   
125                         A current or former player   
126                   Someone who used to attend games   

    When did you last attend a game (home or away)?  \
0                                    29 August 2021   
1                        Last Sunday away at London   
2                          Win over Oldham at home    
3                                 Batley Away match   
4                      Two weeks ago - Batley away.   
..                                              ...   
122                                             NaN   
123                                            2014   
124                                Two weekends ago   
125                                            2010   
126                                            2018   

    What made you first start watching the Sheffield Eagles?  \
0    Student in Sheffield, was a St. Helens fan and...         
1    I had already been watching the game for a few...         
2        Something the whole family could do together          
3                                                 1988         
4                           Mum and Dad were big fans.         
..                                                 ...         
122                                                NaN         
123    Father managing DVS, would go to work with him.         
124                 Free ticket from childrens' school         
125                                       Amateur club         
126                  Playing within Hillsborough Hawks         

    What is your favourite memory of the Sheffield Eagles?   \
0                                 Winning the 1895 Cup        
1                              Challenge Cup win in 98        
2                  Winning league for the second time         
3

In [22]:

questionnaire.rename(columns={
    'Could you provide your postcode? This will help us understand where people are responding from. ':"Postcode",
},inplace=True)

questionnaire


Timestamp                                           Are you:  \
0     9/1/2021 14:05:26                   A player or member of club staff   
1      9/2/2021 9:23:44   A current Sheffield Eagles fan who attends games   
2      9/2/2021 9:25:42   A current Sheffield Eagles fan who attends games   
3     9/2/2021 10:05:39   A current Sheffield Eagles fan who attends games   
4     9/2/2021 10:53:30   A current Sheffield Eagles fan who attends games   
..                  ...                                                ...   
122   9/27/2021 8:00:25                   A player or member of club staff   
123  9/27/2021 10:13:35  A Sheffield Eagles fan who no longer attends g...   
124  9/27/2021 12:35:21   A current Sheffield Eagles fan who attends games   
125  9/27/2021 22:27:20                                   A former player    
126   11/4/2021 0:19:42                   A player or member of club staff   

    When did you first attend a Sheffield Eagles game?  \
0                                           2011- 2021   
1                                          1984 - 1990   
2                                          1991 - 2000   
3                                          1984 - 1990   
4                                          1991 - 2000   
..                                                 ...   
122                                                NaN   
123                                        1991 - 2000   
124                                        1991 - 2000   
125                                        2001 - 2010   
126                                        2001 - 2010   

                          Would you class yourself as:  \
0    Someone who attends games home or away on a re...   
1    Someone who has a season ticket and attends al...   
2    Someone who has a season ticket and attends al...   
3    Someone who attends games home or away on a re...   
4    Someone who has a season ticket and attends al...   
..                                                 ...   
122                         A current or former player   
123                         A current or former player   
124  Someone who has a season ticket and attends al...   
125                         A current or former player   
126                   Someone who used to attend games   

    When did you last attend a game (home or away)?  \
0                                    29 August 2021   
1                        Last Sunday away at London   
2                          Win over Oldham at home    
3                                 Batley Away match   
4                      Two weeks ago - Batley away.   
..                                              ...   
122                                             NaN   
123                                            2014   
124                                Two weekends ago   
125                                            2010   
126                                            2018   

    What made you first start watching the Sheffield Eagles?  \
0    Student in Sheffield, was a St. Helens fan and...         
1    I had already been watching the game for a few...         
2        Something the whole family could do together          
3                                                 1988         
4                           Mum and Dad were big fans.         
..                                                 ...         
122                                                NaN         
123    Father managing DVS, would go to work with him.         
124                 Free ticket from childrens' school         
125                                       Amateur club         
126                  Playing within Hillsborough Hawks         

    What is your favourite memory of the Sheffield Eagles?   \
0                                 Winning the 1895 Cup        
1                              Challenge Cup win in 98        
2                  Winning league for the second time         
3

In [23]:


questionnaire_quantitive = questionnaire[[
    questionnaire.columns[1],
    questionnaire.columns[2],
    questionnaire.columns[3],
    questionnaire.columns[14],
    questionnaire.columns[15],
    questionnaire.columns[16],
    questionnaire.columns[17]
]]
questionnaire_quantitive

Are you:  \
0                     A player or member of club staff   
1     A current Sheffield Eagles fan who attends games   
2     A current Sheffield Eagles fan who attends games   
3     A current Sheffield Eagles fan who attends games   
4     A current Sheffield Eagles fan who attends games   
..                                                 ...   
122                   A player or member of club staff   
123  A Sheffield Eagles fan who no longer attends g...   
124   A current Sheffield Eagles fan who attends games   
125                                   A former player    
126                   A player or member of club staff   

    When did you first attend a Sheffield Eagles game?  \
0                                           2011- 2021   
1                                          1984 - 1990   
2                                          1991 - 2000   
3                                          1984 - 1990   
4                                          1991 - 2000   
..                                                 ...   
122                                                NaN   
123                                        1991 - 2000   
124                                        1991 - 2000   
125                                        2001 - 2010   
126                                        2001 - 2010   

                          Would you class yourself as: Are You?   \
0    Someone who attends games home or away on a re...      Male   
1    Someone who has a season ticket and attends al...      Male   
2    Someone who has a season ticket and attends al...      Male   
3    Someone who attends games home or away on a re...      Male   
4    Someone who has a season ticket and attends al...      Male   
..                                                 ...       ...   
122                         A current or former player      Male   
123                         A current or former player      Male   
124  Someone who has a season ticket and attends al...      Male   
125                         A current or former player      Male   
126                   Someone who used to attend games      Male   

    How old are you?  How would you describe yourself Postcode  
0               25-39                           White      S70  
1               50-59                           White       S2  
2               50-59                           White       S2  
3               50-59                           White      S70  
4               16-25                           White       S5  
..                ...                             ...      ...  
122             60-69                           White     WF10  
123             25-39                           White      HU6  
124             60-69                           White      S11  
125             25-39                           White      S74  
126             25-39  Mixed / Multiple Ethnic Groups      S57  

[127 rows x 7 columns]

In [24]:
questionnaire_quantitive = get_latlongs(questionnaire_quantitive)
questionnaire_quantitive

In [ ]:
questionnaire_quantitive.to_csv("../data/eagles/questionnaire_quantitive.csv")